In [ ]:
import sys
import os
from pathlib import Path

project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from core.backtesting_opt import _Data, Strategy, Backtest
from utils.functions import BlackScholes as bs
import pandas as pd
import numpy as np
import math
from scipy.optimize import root_scalar
from datetime import datetime, date as DateObject,time


In [ ]:
class Oi_Change(Strategy):

    position_id = "001"

    def init(self):
        self.range_offset = 10
        self.strike_step = 50
        self.expiry_type = "weekly"
        self.expiry_range = [9, 16]
        self.legs = []
        self.executed_today = False
        self.last_execution_date = None
        self.ce_oi = 0
        self.pe_oi = 0
        self.ce_data = {"prev_oi": 0, "prev_price": 0}
        self.pe_data = {"prev_oi": 0, "prev_price": 0}
        self.in_pos=0

    def execution_window(self, start_time_str="14:30:00", end_time_str="15:00:00"):
        current_time = pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()

        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date

        start_time = pd.Timestamp(start_time_str).time()
        end_time = pd.Timestamp(end_time_str).time()

        if not (start_time <= current_time <= end_time):
            return False

        if self.executed_today:
            return False

        self.executed_today = True
        return True

    def next(self):
        current_time = pd.Timestamp(self.time).time()
        if self.spot is None or pd.isna(self.spot):
            return

        # Optional: use time window
        # if not self.execution_window():
        #     return

        current_date = pd.Timestamp(self.time).date()
        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date

        atm = round(self.spot / self.strike_step) * self.strike_step
        available_ttes = list(self.tte_to_expiry.keys()) if self.tte_to_expiry else []
        lower, upper = self.expiry_range
        matching_ttes = [tte for tte in available_ttes if lower <= tte <= upper]

        if not matching_ttes:
            print(f"No matching TTEs in range {self.expiry_range}. Found: {available_ttes}")
            return

        valid_tte = min(matching_ttes)
        expiry_date = self.tte_to_expiry[valid_tte]
        self.legs = []
        

        for i in range(-self.range_offset, self.range_offset + 1):
            strike = atm + i * self.strike_step
            for opt_type in ["CE", "PE"]:
                contract = f"NIFTY{pd.Timestamp(expiry_date).strftime('%d%b%y').upper()}{int(strike)}{opt_type}"
                data = self._data.get_ticker_data(contract)
                if data is None:
                    continue
                self.legs.append({
                    "type": opt_type,
                    "strike": strike,
                    "expiry": expiry_date,
                    "contract": contract,
                    "data": data
                })

        if len(self.legs) == 0:
            print("No valid legs found.")
            return

        for leg in self.legs:
            if leg["type"] == "CE":
                self.ce_oi += leg["data"]["open_interest"]
                ce_last_price = leg["data"]["close"]
            elif leg["type"] == "PE":
                self.pe_oi += leg["data"]["open_interest"]
                pe_last_price = leg["data"]["close"]

        # Track price change
        ce_price_chng = ce_last_price - self.ce_data["prev_price"]
        pe_price_chng = pe_last_price - self.pe_data["prev_price"]

        # Track OI percentage change
        pct_chng_ce = ((self.ce_oi - self.ce_data["prev_oi"]) / self.ce_data["prev_oi"]) * 100 if self.ce_data["prev_oi"] else 0
        pct_chng_pe = ((self.pe_oi - self.pe_data["prev_oi"]) / self.pe_data["prev_oi"]) * 100 if self.pe_data["prev_oi"] else 0

        # Behavior flags
        ce_oi_up = pct_chng_ce > 50
        ce_oi_down = pct_chng_ce < -30
        pe_oi_up = pct_chng_pe > 50
        pe_oi_down = pct_chng_pe < -30
        ce_price_up = ce_price_chng > 0
        ce_price_down = ce_price_chng < 0
        pe_price_up = pe_price_chng > 0
        pe_price_down = pe_price_chng < 0
        price_neutral_threshold = 30  # percent

        ce_price_neutral = abs(ce_price_chng) <= price_neutral_threshold
        pe_price_neutral = abs(pe_price_chng) <= price_neutral_threshold
        pe_oi_neutral=pct_chng_pe < 30

        # Signal map
        signals = {
            "long_straddle": lambda: ce_oi_up and pe_oi_up and ce_price_down and pe_price_down,
            "short_straddle": lambda: ce_oi_up and pe_oi_up and ce_price_up and pe_price_up,

            "call_long_buildup": lambda: ce_oi_up and ce_price_up,
            "call_short_buildup": lambda: ce_oi_up and ce_price_down,
            "call_long_liquidation": lambda: ce_oi_down and ce_price_down,
            "call_short_covering": lambda: ce_oi_down and ce_price_up,

            "put_long_buildup": lambda: pe_oi_up and pe_price_up,
            "put_short_buildup": lambda: pe_oi_up and pe_price_down,
            "put_long_liquidation": lambda: pe_oi_down and pe_price_down,
            "put_short_covering": lambda: pe_oi_down and pe_price_up,

            "long_call": lambda: ce_oi_up and ce_price_up,  # strongly bullish
            "long_put": lambda: pe_oi_up and pe_price_up,   # strongly bearish
            "short_put": lambda: pe_oi_up and pe_price_down,  # bullish but not very strong
            "bull_call_spread": lambda: ce_oi_up and ce_price_up and pe_price_neutral,
            "short_strangle": lambda: ce_oi_up and ce_price_down and pe_oi_neutral,
            # "protective_put": lambda: pe_oi_up and pe_price_up and holding_spot_position,
        }


        # Execute signals
        for signal_name, condition in signals.items():
            if condition():
                print(f"Signal Detected: {signal_name.upper()}")
                self.execute_strategy(signal_name, atm, expiry_date)

        # Updating next tick
        self.ce_data["prev_oi"] = self.ce_oi
        self.ce_data["prev_price"] = ce_last_price
        self.pe_data["prev_oi"] = self.pe_oi
        self.pe_data["prev_price"] = pe_last_price

        print(f"CE OI: {self.ce_oi}, PE OI: {self.pe_oi}")

        if current_time >= time(15,15) and self.in_pos!=0:
            print("close all positions")
            # trades=self.all_trades_or_none
            # print(trades) 
            print(self.active_trades)
            # for trade in self.active_trades:
            #     print(trade)

            print("-"*100)
        if current_time >= time(15,15):
            self.ce_oi = 0
            self.pe_oi = 0

    def execute_strategy(self, signal, atm, expiry):
        strategy_id = self.__class__.__name__
        leg_ce = f"NIFTY{pd.Timestamp(expiry).strftime('%d%b%y').upper()}{atm}CE"
        leg_pe = f"NIFTY{pd.Timestamp(expiry).strftime('%d%b%y').upper()}{atm}PE"

        if signal == "long_straddle" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="long_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=50.0, tag=signal)
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="long_pe", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=50.0, tag=signal)
            self.in_pos += 1

        elif signal == "short_straddle" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="short_ce", ticker=leg_ce, quantity=1, stop_loss=30.0, take_profit=50.0, tag=signal)
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="short_pe", ticker=leg_pe, quantity=1, stop_loss=30.0, take_profit=50.0, tag=signal)
            self.in_pos += 1

        elif signal == "call_long_buildup" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="long_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "call_short_buildup" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="short_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "call_long_liquidation" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="liquidate_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "call_short_covering" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="cover_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "put_long_buildup" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="long_pe", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "put_short_buildup" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="short_pe", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "put_long_liquidation" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="liquidate_pe", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "put_short_covering" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="cover_pe", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "long_call" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="bullish_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=50.0, tag=signal)
            self.in_pos += 1

        elif signal == "long_put" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="bearish_pe", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=50.0, tag=signal)
            self.in_pos += 1

        elif signal == "short_put" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="bullish_put", ticker=leg_pe, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        elif signal == "bull_call_spread" and self.in_pos == 0:
            self.buy(strategy_id=strategy_id, position_id=self.position_id, leg_id="bull_call_buy", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            otm_ce_strike = str(int(atm) + 100)
            leg_ce_otm = f"NIFTY{pd.Timestamp(expiry).strftime('%d%b%y').upper()}{otm_ce_strike}CE"
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="bull_call_sell", ticker=leg_ce_otm, quantity=1, stop_loss=20.0, take_profit=30.0, tag=signal)
            self.in_pos += 1

        elif signal == "short_strangle" and self.in_pos == 0:
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="strangle_ce", ticker=leg_ce, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            otm_pe_strike = str(int(atm) - 100)
            leg_pe_otm = f"NIFTY{pd.Timestamp(expiry).strftime('%d%b%y').upper()}{otm_pe_strike}PE"
            self.sell(strategy_id=strategy_id, position_id=self.position_id, leg_id="strangle_pe", ticker=leg_pe_otm, quantity=1, stop_loss=20.0, take_profit=40.0, tag=signal)
            self.in_pos += 1

        else:
            print(f"[{signal}] No defined strategy for this signal.")

        print(f"Executed {signal.upper()} using ATM strike {atm}")



In [25]:
# db_path = "nifty_1min_desiquant.duckdb"
db_path = "nifty_opt_icici_1min.duckdb"

bt = Backtest(db_path=db_path, strategy=Oi_Change, cash=10000000, commission_per_contract=0.65, option_multiplier=75)
stats = bt.run(start_date="2022-01-01", end_date="2022-01-30")
print(stats)

Identifying tables in date range...
Found 19 tables out of 966 total tables with data in range


Backtesting Options Strategy:   0%|          | 0/19 [00:00<?, ?it/s]

pct_chng_ce: 0, pct_chng_pe: 0
price_chng_ce: 22.100000381469727, price_chng_pe: 507.8500061035156
CE OI: 126950.0, PE OI: 71600.0
pct_chng_ce: 69.791259765625, pct_chng_pe: 91.34078216552734
price_chng_ce: -0.6000003814697266, price_chng_pe: 0.0
Signal Detected: CALL_SHORT_BUILDUP
Executed CALL_SHORT_BUILDUP using ATM strike 17450
CE OI: 215550.0, PE OI: 137000.0
exec_price: 183.75, order: NIFTY13JAN2217450CE, size: -1
pct_chng_ce: 60.171653747558594, pct_chng_pe: 56.42335891723633
price_chng_ce: 0.14999961853027344, price_chng_pe: 0.0
Signal Detected: CALL_LONG_BUILDUP
[call_long_buildup] No defined strategy for this signal.
Executed CALL_LONG_BUILDUP using ATM strike 17450
Signal Detected: LONG_CALL
[long_call] No defined strategy for this signal.
Executed LONG_CALL using ATM strike 17450
Signal Detected: BULL_CALL_SPREAD
[bull_call_spread] No defined strategy for this signal.
Executed BULL_CALL_SPREAD using ATM strike 17450
CE OI: 345250.0, PE OI: 214300.0
pct_chng_ce: 21.274438858

Backtesting Options Strategy:   5%|▌         | 1/19 [00:00<00:08,  2.13it/s]

pct_chng_ce: 4.759816646575928, pct_chng_pe: 5.169500827789307
price_chng_ce: -4.949999809265137, price_chng_pe: 0.0
CE OI: 1911500.0, PE OI: 1462750.0
pct_chng_ce: 1.8545644283294678, pct_chng_pe: 2.273115634918213
price_chng_ce: 0.25, price_chng_pe: 0.0
CE OI: 1946950.0, PE OI: 1496000.0
pct_chng_ce: 1.7642980813980103, pct_chng_pe: 1.3602941036224365
price_chng_ce: 0.19999980926513672, price_chng_pe: 0.0
CE OI: 1981300.0, PE OI: 1516350.0
pct_chng_ce: 1.0472921133041382, pct_chng_pe: 0.9331618547439575
price_chng_ce: 5.149999618530273, price_chng_pe: 0.0
CE OI: 2002050.0, PE OI: 1530500.0
pct_chng_ce: 2.255188465118408, pct_chng_pe: 1.6203854084014893
price_chng_ce: -0.5999984741210938, price_chng_pe: -9.29998779296875
CE OI: 2047200.0, PE OI: 1555300.0
pct_chng_ce: 1.0282336473464966, pct_chng_pe: 0.7169035077095032
price_chng_ce: 0.0, price_chng_pe: 0.0
CE OI: 2068250.0, PE OI: 1566450.0
pct_chng_ce: 1.2232563495635986, pct_chng_pe: 0.9799227118492126
price_chng_ce: -4.40000057220

...

Backtesting Options Strategy:  89%|████████▉ | 17/19 [00:07<00:00,  2.30it/s]

pct_chng_ce: 1.1143158674240112, pct_chng_pe: 1.360707402229309
price_chng_ce: 11.949996948242188, price_chng_pe: -29.54998779296875
CE OI: 2436400.0, PE OI: 1690950.0
pct_chng_ce: 2.0193727016448975, pct_chng_pe: 3.790768623352051
price_chng_ce: -4.399993896484375, price_chng_pe: 13.0999755859375
CE OI: 2485600.0, PE OI: 1755050.0
pct_chng_ce: 3.7254586219787598, pct_chng_pe: 5.27620267868042
price_chng_ce: 0.3499908447265625, price_chng_pe: -2.0999755859375
CE OI: 2578200.0, PE OI: 1847650.0
pct_chng_ce: 1.5359553098678589, pct_chng_pe: 2.6574296951293945
price_chng_ce: -0.75, price_chng_pe: 9.54998779296875
CE OI: 2617800.0, PE OI: 1896750.0
pct_chng_ce: 1.2777905464172363, pct_chng_pe: 2.5438249111175537
price_chng_ce: 1.75, price_chng_pe: 0.0
CE OI: 2651250.0, PE OI: 1945000.0
pct_chng_ce: 1.133427619934082, pct_chng_pe: 1.4318766593933105
price_chng_ce: 0.0, price_chng_pe: 0.0
CE OI: 2681300.0, PE OI: 1972850.0
pct_chng_ce: 1.36687433719635, pct_chng_pe: 1.540917992591858
price_c

Backtesting Options Strategy:  95%|█████████▍| 18/19 [00:07<00:00,  2.29it/s]

pct_chng_ce: 0.7917773127555847, pct_chng_pe: 0.7231605648994446
price_chng_ce: -12.899993896484375, price_chng_pe: 80.0
CE OI: 1024750.0, PE OI: 1037650.0
pct_chng_ce: 0.814832866191864, pct_chng_pe: 0.6167783141136169
price_chng_ce: 1.5, price_chng_pe: 0.0
CE OI: 1033100.0, PE OI: 1044050.0
pct_chng_ce: 1.340625286102295, pct_chng_pe: 1.1828935146331787
price_chng_ce: 11.399993896484375, price_chng_pe: -80.0
CE OI: 1046950.0, PE OI: 1056400.0
pct_chng_ce: 1.2130475044250488, pct_chng_pe: 1.755963683128357
price_chng_ce: 0.0, price_chng_pe: 0.0
CE OI: 1059650.0, PE OI: 1074950.0
pct_chng_ce: 0.7785589694976807, pct_chng_pe: 0.6884041428565979
price_chng_ce: -6.25, price_chng_pe: 0.0
CE OI: 1067900.0, PE OI: 1082350.0
pct_chng_ce: 0.8053188323974609, pct_chng_pe: 0.7021757960319519
price_chng_ce: 0.0, price_chng_pe: 0.0
CE OI: 1076500.0, PE OI: 1089950.0
pct_chng_ce: 0.6549001336097717, pct_chng_pe: 0.532134473323822
price_chng_ce: 0.0, price_chng_pe: 0.0
CE OI: 1083550.0, PE OI: 10957

pct_chng_ce: 0.8722881078720093, pct_chng_pe: 1.1994458436965942
price_chng_ce: -6.850006103515625, price_chng_pe: 16.5999755859375
CE OI: 1804000.0, PE OI: 1497600.0
pct_chng_ce: 1.072616457939148, pct_chng_pe: 2.2903311252593994
price_chng_ce: -2.399993896484375, price_chng_pe: 3.300048828125
CE OI: 1823350.0, PE OI: 1531900.0
pct_chng_ce: 0.8912167549133301, pct_chng_pe: 0.7996605634689331
price_chng_ce: -0.5500030517578125, price_chng_pe: 7.5
CE OI: 1839600.0, PE OI: 1544150.0
pct_chng_ce: 0.6659056544303894, pct_chng_pe: 1.2531166076660156
price_chng_ce: 3.6999969482421875, price_chng_pe: -21.6500244140625
CE OI: 1851850.0, PE OI: 1563500.0
pct_chng_ce: 1.56330144405365, pct_chng_pe: 1.8100416660308838
price_chng_ce: -7.25, price_chng_pe: 27.9000244140625
CE OI: 1880800.0, PE OI: 1591800.0
pct_chng_ce: 1.105912446975708, pct_chng_pe: 1.042844533920288
price_chng_ce: 4.9499969482421875, price_chng_pe: 0.0
CE OI: 1901600.0, PE OI: 1608400.0
pct_chng_ce: 1.8063734769821167, pct_chng_

In [9]:
from core.wfo import WalkForwardOptimizer

constraint = lambda x: x['upper_threshold'] > x['lower_threshold']


wfo = WalkForwardOptimizer(
    strategy=IV_Slope,
    optimization_params={
        # 'upper_threshold': [-0.15 ,-0.1, 0.1, 0.15, 0.2, 0.25, 0.3],
        # 'lower_threshold': [-0.15 ,-0.1 ,0, 0.05, 0.1, 0.15, 0.2]
        'upper_threshold': [0.1, 0.15, 0.2, 0.25],
        'lower_threshold': [ 0.1, 0.15, 0.2]
    },
    # constraint=constraint,
    maximize='Sharpe Ratio',
)

In [ ]:
# def optimize_stock(self, stock, timeframe, db_path, exchange = None, training_candles = 2000, testing_candles = 200):
    
wfo.optimize_stock(
    stock='NIFTY',
    timeframe='1min',
    db_path='../core/nifty_1min_desiquant.duckdb',
    exchange='NSE',
    training_candles=6000,
    testing_candles=3000,
    start_date='2022-01-01',
    end_date='2022-12-31',
    # method='sambo',
    # max_tries=10
)

In [ ]:
bt.tear_sheet()